# Advent of Code 2024 - J12

In [1]:
def read_input(kind):
    assert kind in ('input', 'example', 'example1', 'example2'), '"kind" must be "input" or "example" or "example1" or "example2"'
    garden = []
    with open(kind) as f:
        for line in f:
            garden.append(line.strip())
    return garden

In [2]:
def print_garden(garden):
    colors = [
        '\033[30m', '\033[31m', '\033[32m', '\033[33m',
        '\033[34m', '\033[35m', '\033[36m', '\033[37m',
        '\033[90m', '\033[91m', '\033[92m', '\033[93m',
        '\033[94m', '\033[95m', '\033[96m', '\033[97m']
    letters = {}
    current = 0
    for line in garden:
        l = ''
        for c in line:
            if c not in letters:
                letters[c] = colors[current]
                current = (current + 1)%len(colors)
            l += (letters[c]+c)
        print(l)

In [3]:
print_garden(read_input("example1"))

AAAA
BBCD
BBCC
EEEC


In [4]:
print_garden(read_input("example2"))

OOOOO
OXOXO
OOOOO
OXOXO
OOOOO


In [5]:
print_garden(read_input("example"))

RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE


In [6]:
print_garden(read_input("input"))

IIIIIIIIIIIIIIIUUUUUUUUUUUUUUUUUUUSSAAAAAAAAAAAOFFFFFFFFFFFFCCCNNNKKKKKKKKKKKGGGGGGGHHHHHHHHCCRXXXXXXXXIIIILKLLLLLLLLLLLLLLLLFTTTTTTTTTTWWWW
IIAIIIYIIIIIIIIIUUUUUUUUUUUUUUUUUUUSUAAAAAAAAAAFFFFFFFFFEFFCCCCNNNKKKKKKKKGGGGGGNGGGHHHHHHHCCCXXXXXXXXXIIIILLLLLLLLLLLLLLLLLLFTTTTTTTTTTWWXW
IIAIIIYYYYYYYYYYYUUUUUUUUUUUUUUUUUUUUAAAAAAAAAAFFEEEFFEEEENNNNNNNNKKKKKKGGGGGGGGGGGGGGGHHHHCCCCXCXXXXXIIIILLLLLLLLLLLLLLLLLLLFTTTTTTTTTWWXXW
IAAAIIYYYYYYYYYYLUUUUUUUUUHHUUUUUUUUAAAAAAAAAAAFEEEEEFEEEENNNNNNNNNKKKKKKGGGGGGGGGGGGGGHHHCCCCCCXXXXXXIMIMLLMLLLLLLLLLLLLLLLFFFFFTTTTTTXXXXX
AAAIIIYYYYYYYYYYUUUUUUUUUUHHHHHHUUUUUUUAAAAAAAAAEEEEEEEEEEEWNNNNZZZZZKKKGGGGGGGGGGGGGGGGGCCCCCCCCXXXXXIMMMLLMLLLLLLLLLLLLLLLLFFTTTTTTTTXXXXX
AAAIIYYYYYYYYYYYYUUUUUUUUHHHHHHHHHHUUUAAAAAAAAAAEEEEEEEEEEENNNNZZZZZZZZKKKGGGGGGGGGGGGGGWCCCCCCCXXXXXXMMMMLMMMMLLLLLLLLLLLLLFFFTTTTTTTTXXXXQ
AAAIIIYRRYYYYYYYYUUUUHHHHHHHHHHHHHHAAUAAAAAAAAAEEEOEEEEEEEEENNNNNZZZKKKKKKKGGGGGGGGGGGGGWCCCCCCCRXXXJJMMMMMMMMMLLLLLLLLLLLLFFRTTTTTXXXXXXXXX
AAAAIIARRRYYY

## First part

Multiply area by perimeter for the price of each _garden plot_ and add all prices.

example1: 4×10 + 4×8 + 4×10 + 1×4 + 3×8 = **140**\
example2: 21×36 + (1×4)×4 = **772**\
example: **1930**